# LSTM

In [1]:
import jieba
import logging
import numpy as np
from gensim.models import word2vec
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors

/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-pack

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

## jieab tokenize

In [2]:
def jieba_tokenizer(content):
    # load stopwords set
    stopword_set = set()
    with open('jieba_dict/stopwords.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip(' \n'))

    if str(content)=='nan':
        return ''
    words = jieba.cut(content.replace(' ',''))
    return [word for word in words if word not in stopword_set]

In [4]:
def word2token(word,w2v_model):

    # Retrieve the weights from the model. This is used for initializing the weights
    # in a Keras Embedding layer later
    w2v_weights = w2v_model.wv.vectors
    vocab_size, embedding_size = w2v_weights.shape

    try:
        return w2v_model.wv.vocab[word].index
    # If word is not in index return 0. I realize this means that this
    # is the same as the word of index 0 (i.e. most frequent word), but 0s
    # will be padded later anyway by the embedding layer (which also
    # seems dirty but I couldn't find a better solution right now)
    except KeyError:
        return 0
    
def token2word(token):
    return w2v_model.wv.index2word[token]

## ckip tokenize

In [ ]:
def ckip_tokenizer(content):
#     if str(content)=='nan':
#         return ''
    word_s = ws([content])
    return word_s

## predict

In [7]:
def suicide_predict(text):
    model = load_model('lstm_model.h5')

    # jieba custom setting.
    jieba.set_dictionary('jieba_dict/dict.txt.big')
    text_token = jieba_tokenizer(text)
    
    
    w2v_model = KeyedVectors.load("w2v_model.wv", mmap='r')   
    
    for i in range(len(text_token)):
        text_token[i] = word2token(text_token[i],w2v_model)

    text_token = pad_sequences([text_token], maxlen=250, padding='post', value=0)

    score = model.predict(text_token)
    if score[0][0] > score[0][1]:
        results = 0
    else:
        results = 1

    return results

In [10]:
suicide_predict("也許雨季來臨前就開始了吧什麼都不想做只覺得好累好累考完駕照雖然開心但更覺得鬆了一口氣終於不用在約定時間跟人見面要讀的paper越積越多進度仍是零雖然可以隨便做做但還是想試著做好想試著做好卻一直無法專心讀著下禮拜要上臺介紹的散文集羨慕那些可以有辦法寫的人而我寫的東西永遠詞不達義--")

Building prefix dict from /home/ubuntu-gpu/Project_Depression/jieba_dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u156e2db6fd044c50a8c659fc2a66f9b6.cache
Loading model cost 1.295 seconds.
Prefix dict has been built succesfully.
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


0

# BERT

In [1]:
import torch
from torch.utils.data import Dataset,random_split
from transformers import BertTokenizer, BertForMaskedLM, BertForSequenceClassification
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
PRETRAINED_MODEL_NAME = "bert-base-chinese"

/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-pack

In [6]:
# load bert entire model
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
model = torch.load('bert_514.h5')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [85]:
print("""
name      module
--------------------""")

for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print("{:10}{}".format(name,n) )
    else:
        print("{:10} {}".format(name, module))


name      module
--------------------
bert      embeddings
bert      encoder
bert      pooler
dropout    Dropout(p=0.1, inplace=False)
classifier Linear(in_features=768, out_features=2, bias=True)


In [5]:
# load bert state dict
NUM_LABELS = 2
model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)
model.load_state_dict(torch.load('bert_542.h5'))
model.eval()

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [87]:
text = "好想死"

In [88]:
class_names = ['negative', 'positive']

encoded_text = tokenizer.encode_plus(
    text,
    max_length=300,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [89]:
input_ids = encoded_text['input_ids'].to(device)
attention_mask = encoded_text['attention_mask'].to(device)
output = model(input_ids, attention_mask)
_, prediction = torch.max(output[0][0], dim=0)

print(f'Review text: {text}\n')
print(f'Sentiment  : {class_names[prediction]}')

Review text: 好想死

Sentiment  : negative


## AutoLabel_BERT

In [30]:
import pandas as pd

In [31]:
data = pd.read_csv('dcard_unlabeled.csv')

In [33]:
class_names = ['0', '1']
label = []

with open('dcard_unlabeled.csv', 'w') as file:
    for i in range(len(data['Content'])):
        text = data['Content'][i]

        encoded_text = tokenizer.encode_plus(
            text,
            max_length=300,
            add_special_tokens=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].to(device)
        attention_mask = encoded_text['attention_mask'].to(device)
        output = model(input_ids, attention_mask)
        _, prediction = torch.max(output[0][0], dim=0)

        
        label.append(class_names[prediction])
        print(i, label[i], '\n')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a speci

0 0 



Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1 1 

2 1 



Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

1685 0 

1686 1 

1687 1 



Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

1688 1 

1689 1 

1690 1 



Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

1691 1 

1692 0 

1693 0 

1694 1 



In [36]:
data.insert(1, "Label", label)

In [37]:
data

,Content,Label
0,想知道有沒有人在低潮的時候有雙重情緒的感覺最近幾乎處在自殺邊緣很難具體描述這種感覺如果真的要...,0
1,不知為甚麼有時突然情緒就低落，然後就會想著真想被車子撞死，也不知為何會這樣想，但是當人真的很...,1
2,星期天深夜，我吞藥自殺了18顆的樂力靜與一罐500ml的啤酒意識越來越模糊，只記得警察似乎來...,1
3,18年來，我歷經了多少的事情，受過多少的唾棄，受過多少的嘲笑，受過多少的霸淩，不管言語或者肢...,1
4,ps:看到之後真的心情很糟，作為守護民眾的公務人員竟然做出這種事。消防員私抄17歲報案少女電...,0
...,...,...
1690,你好，我是2020的你，今天是2020的第一天。我回顧了前兩年的時光膠囊，第一年的我責備，第...,1
1691,用時間去證明一切，當彼此都有人生目標時，千萬不要因為一個人告白去放棄自己的一切，我的愛情觀不...,1
1692,早就看以偷拍公眾人物私生活爆料維生的狗仔不爽很久了然後這個社會從來沒有人反抗過這件事？酷炫是...,0
1693,19 年的妳遇到了各種大大小小的事情想必 2020 的妳只會遇到更多的起伏跟挑戰希望這一年的...,0


In [38]:
data.to_csv('labeled.csv', index=False)